In [ ]:
import xarray as xr
import rioxarray as riox
import pandas as pd
import os
import re
import numpy as np
import panel as pn
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
from cartopy import crs
import param as pm
import geopandas as gpd
import glob
from fnmatch import fnmatch
from shapely.geometry import Polygon
from distributed import Client, LocalCluster
#pn.extension()
hv.extension('bokeh')
pn.param.ParamMethod.loading_indicator = True

In [ ]:
cluster_loc = None

if cluster_loc == None:
    pass
elif cluster_loc == 'local':
    clust = LocalCluster(n_workers=8, threads_per_worker=2)
    client = Client(clust)
    display(client)
elif cluster_loc == 'coiled':
    import coiled
    coiled.create_software_environment(
    name="py_geo",
    conda="py_geo_environment.yaml")
    clust = coiled.Cluster(
        name="avhrr_cog_coiled",
        software="py_geo",
        n_workers=32, # 200 and then 1 with worker_cpu=128 USE SPOT
        worker_cpu=2,
        backend_options={"spot": True})
    client = Client(clust)
    display(client)
elif cluster_loc == 'hpc':
    import dask_jobqueue as jq
    import dask
    dask.config.set({'distributed.dashboard.link': '/user/{USER}/proxy/{port}/status'})
    partition='short'#,debug,mem,mem-low'
    num_processes = 12
    num_threads_per_processes = 2
    mem = 0.8*num_processes*num_threads_per_processes#*1.25
    n_cores_per_job = num_processes*num_threads_per_processes
    clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            interface='ib0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            walltime='02:00:00',
                            job_extra=["--output=/dev/null","--error=/dev/null"])
    client=Client(clust)
    display(client)

In [ ]:
gdf_ground_points_all = gpd.read_file('ground_data/cper_pdog_points_2021Sept.shp')

In [ ]:
for idx, pasture in enumerate(['5W', '22W', '22E', '29-30', 'CN']):
    if idx == 0:
        df_bboxes = pd.read_csv('train_tiles/train_bboxes_' + pasture + '.csv')
    else:
        df_bboxes = pd.concat([df_bboxes, pd.read_csv('train_tiles/train_bboxes_' + pasture + '.csv')])

In [ ]:
df_bboxes = df_bboxes[df_bboxes['Digitize'] == 1].reset_index(drop=True)
df_bboxes['path_pre'] = df_bboxes.apply(lambda x: os.path.join('train_tiles',
                                                           x['Pasture'] + '_' + x['Type'],
                                                           x['Pasture'] + '_' + x['Tile'] + '_'), axis=1)

In [ ]:
#df_bboxes[['All', 'Lauren', 'David', 'Sean', 'Nick']] = 0
df_bboxes.loc[df_bboxes[df_bboxes['Type'] == 'random'].groupby(
    'Pasture').sample(n=2, random_state=123).index, 'trainer'] = 'All'
df_bboxes.loc[df_bboxes[df_bboxes['Type'] == 'burrows_active'].groupby(
    'Pasture').sample(n=1, random_state=223).index, 'trainer'] = 'All'

In [ ]:
df_bboxes_sub = df_bboxes.loc[df_bboxes['trainer'] != 'All'].sample(frac=1, random_state=323)

In [ ]:
df_bboxes_sub.iloc[:30, df_bboxes_sub.columns.get_loc('trainer')] = 'Lauren'
df_bboxes_sub.iloc[30:60, df_bboxes_sub.columns.get_loc('trainer')] = 'David'
df_bboxes_sub.iloc[60:90, df_bboxes_sub.columns.get_loc('trainer')] = 'Sean'
df_bboxes_sub.iloc[90:, df_bboxes_sub.columns.get_loc('trainer')] = 'Nick'

In [ ]:
df_bboxes_fnl = pd.concat([df_bboxes[df_bboxes['trainer'] == 'All'], df_bboxes_sub])
df_bboxes_fnl.loc[df_bboxes_fnl['Poly_ID'].isnull(), 'Poly_ID'] = 'None'
type_dict = {'burrows_active': 0, 'random': 1}
print(type_dict)

In [ ]:
class TrainApp(pm.Parameterized):
    window_size_m = 30
    slider_width=150
    type_dict = {'burrows_active': 0, 'random': 1}
    
    delim_burrows = gpd.GeoDataFrame(columns=['Comment', 'Tile', 'Trainer', 'geometry'])
    
    select_trainer = pn.widgets.Select(options=list(df_bboxes_fnl[df_bboxes_fnl['trainer'] != 'All']['trainer'].unique()),
                                       name='Select trainer', width=slider_width, size=1)
    
    radio_all = pn.widgets.Checkbox(name='Group training?', value=True, width=slider_width)
    
    df_bboxes_all = df_bboxes_fnl.copy(deep=True)
    if radio_all.value:
        df_bboxes = df_bboxes_all[df_bboxes_all['trainer'] == 'All'].sort_values('Type',
                                                                                 key=lambda x: x.map(type_dict))
    else:
        df_bboxes = df_bboxes_all[df_bboxes_all['trainer'] == select_trainer.value].sort_values('Type',
                                                                                                key=lambda x: x.map(type_dict))
    #tile_file_list = glob.glob('train_tiles/' + pasture + '**/*.tif')
    
    gdf_ground_points = gdf_ground_points_all#[gdf_ground_points_all['Pasture'] == pasture]
    gdf_burrows = gdf_ground_points[gdf_ground_points['is_burrow'] == 1]
    gdf_other = gdf_ground_points[gdf_ground_points['is_burrow'] == 0]

    ndvi_alpha = pn.widgets.FloatSlider(start=0.0, end=1.0, step=0.05, value=0.0, width=slider_width, name='Transparency')
    ndvi_range = pn.widgets.RangeSlider(start=0.0, end=0.50, step=0.01, value=(0.0, 0.20), width=slider_width, name='Color range')

    terrain_alpha = pn.widgets.FloatSlider(start=0.0, end=1.0, step=0.05, value=0.0, width=slider_width, name='Transparency')
    tpi_alpha = pn.widgets.FloatSlider(start=0.0, end=0.5, step=0.05, value=0.0, width=slider_width, name='Transparency')
    
    terrain_range = pn.widgets.RangeSlider(start=-0.50, end=1.0, step=0.01, value=(-0.10, 0.40), width=slider_width, name='Color range')   

    ground_alpha = pn.widgets.FloatSlider(start=0.0, end=1.0, step=0.05, value=0.0, 
                                          width=slider_width, name='Transparency')
    
    #tile_list = df_bboxes['ID'].sort_values().to_list()
    
    tile_list = df_bboxes['ID'].to_list()
    tile_list_dict = {k: v for (k, v) in zip(df_bboxes['ID'].to_list(), 
                                             df_bboxes['ID'].index.values)}
    
    save_polys = pm.Action(lambda x: x.param.trigger('save_polys'), label='Save polygons')
    #select_tile = pm.Selector(objects=tile_list.param, default=tile_list.param.value[0])
    select_tile = pn.widgets.Select(options=tile_list, name='Select tile', width=slider_width, size=5)
    
    map_opts = dict(projection=crs.UTM(13), responsive=False, width=700, height=700, xaxis=None, yaxis=None,
                     padding=0, tools=['pan', 'box_zoom'], framewise=False,
                     active_tools=['wheel_zoom'], toolbar='left')
    
    map_args = dict(rasterize=True, project=False, dynamic=True)
    
    draw_poly_opts = dict(fill_color='yellow', fill_alpha=0.10, line_color='yellow', line_width=2)  
    
    saved_poly_opts = dict(fill_color='yellow', fill_alpha=0.20, line_color='yellow', line_width=2)
    
    burrow_poly_dict = {k: hv.Polygons([]) for k in df_bboxes_all['ID']}
    
    def __init__(self, **params):
        super(TrainApp, self).__init__(**params)
        
        #self.i = int(self.select_tile.value.split('_')[-1])
        #self.select_tile = pn.widgets.Select(options=self.tile_list, name='Select tile', width=self.slider_width, size=5)
        self.i = self.tile_list_dict[self.select_tile.value]
        self.burrows_annotate = None
        self.burrows_layout = None
        self._burrows = hv.Polygons([])
        
        self.tpi_alpha.value = self.terrain_alpha.value / 2.0
        
        #self.tiles = [x for x in self.tile_file_list if fnmatch(x.split('/')[-1], self.pasture + '_' + self.select_tile.value + '_*.tif')]
        self.rgb = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'rgb.tif').astype('int')
        self.rgb_img = self.rgb.hvplot.rgb(x='x', y='y', bands='band',
                                               **self.map_args).opts(**self.map_opts)
        self.rangexy = hv.streams.RangeXY(source=self.rgb_img)


        self.ndvi = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'ndvi.tif').squeeze()
        self.ndvi_img = self.ndvi.hvplot.image(x='x', y='y', alpha=0.5,
                                                         **self.map_args).opts(cmap='viridis',
                                                                               colorbar=False,
                                                                               **self.map_opts)

        self.shade = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'shade.tif').squeeze()
        self.tpi = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'tpi.tif').squeeze()
        
        self.shade_img = self.shade.hvplot.image(x='x', y='y', 
                                     **self.map_args).opts(cmap='gray',
                                                           clim=(210, 225),
                                                           colorbar=False,
                                                           **self.map_opts)
        self.tpi_img = self.tpi.hvplot.image(x='x', y='y',
                                                    **self.map_args).opts(cmap='turbo',
                                                                          colorbar=False,
                                                                          **self.map_opts)

        
        self.minx, self.miny, self.maxx, self.maxy = self.df_bboxes.loc[self.i, ['min_x', 'min_y', 'max_x', 'max_y']].values
        
        self.poly_tmp = hv.Polygons(hv.Bounds((self.minx, self.miny, self.maxx, self.maxy))).opts('Polygons',
                                                                                                  line_color='red',
                                                                                                  fill_color=None, 
                                                                                                  **self.map_opts)
        
        self.p_ground_burrows = hv.Points(data=list(zip(self.gdf_burrows.geometry.x.values,
                                               self.gdf_burrows.geometry.y.values))).opts(color='red', 
                                                                                     marker='o',
                                                                                     size=12,
                                                                                     framewise=False)
        self.p_ground_other = hv.Points(data=list(zip(self.gdf_other.geometry.x.values,
                                                   self.gdf_other.geometry.y.values))).opts(color='orange', 
                                                                                       marker='x',
                                                                                       line_width=4,
                                                                                       size=15,
                                                                                       framewise=False)
        
    @pm.depends('terrain_alpha.param', watch=True)
    def calc_tpi_alpha(self):
        self.tpi_alpha.value = self.terrain_alpha.value / 2.0
    
    def show_rgb(self):
        return self.rgb_img
    
    def show_ndvi(self):
        return self.ndvi_img.apply.opts(alpha=self.ndvi_alpha.param.value,
                                        clim=self.ndvi_range.param.value)

    def show_terrain(self):
        return (self.shade_img.apply.opts(alpha=self.terrain_alpha.param.value) * self.tpi_img.apply.opts(alpha=self.tpi_alpha.param.value,
                                                                                                      clim=self.terrain_range.param.value))
    
    
    @pm.depends('select_trainer.param', 'radio_all.param', watch=True)
    def update_tilelist(self):
        if self.radio_all.value:
            self.df_bboxes = self.df_bboxes_all[self.df_bboxes_all['trainer'] == 'All'].sort_values('Type', key=lambda x: x.map(self.type_dict))
        else:
            self.df_bboxes = self.df_bboxes_all[self.df_bboxes_all['trainer'] == self.select_trainer.value].sort_values('Type',
                                                                                                                        key=lambda x: x.map(self.type_dict))
        self.tile_list = self.df_bboxes['ID'].to_list()
        self.tile_list_dict = {k: v for (k, v) in zip(self.df_bboxes['ID'].to_list(), 
                                                      self.df_bboxes['ID'].index.values)}
        #self.select_tile = pn.widgets.Select(options=self.tile_list, name='Select tile', width=self.slider_width, size=5)
        self.i = self.tile_list_dict[self.tile_list[0]]
        self.select_tile.options = self.tile_list
        self.select_tile.set_default = self.tile_list[0]
        #return self.select_tile

    @pm.depends('save_polys')
    def save_data(self):
        if self.burrows_annotate is not None:
            if len(self.burrows_annotate.annotated.data) > 0:
                self.burrow_poly_dict[self.df_bboxes.loc[self.i, 'ID']] = self.burrows_annotate.annotated
                for i in range(len(self.burrow_poly_dict[self.df_bboxes.loc[self.i, 'ID']].data)):
                    self.burrow_poly_dict[
                        self.df_bboxes.loc[self.i, 'ID']].data[i]['Tile'] = self.tile_list[self.df_bboxes.index.get_loc(self.i)]
                    self.burrow_poly_dict[
                        self.df_bboxes.loc[self.i, 'ID']].data[i]['Trainer'] = self.select_trainer.value
                tile_burrows_tmp = gpd.GeoDataFrame(data=self.burrow_poly_dict[self.df_bboxes.loc[self.i, 'ID']].data)
                tile_burrows_tmp.set_geometry(tile_burrows_tmp.apply(lambda row: Polygon(zip(row['x'], row['y'])), axis=1), inplace=True)
                tile_burrows_tmp.set_crs(epsg='32613', inplace=True)
                tile_burrows_tmp = tile_burrows_tmp.drop(columns=['x', 'y'])
                self.delim_burrows = self.delim_burrows.append(tile_burrows_tmp).drop_duplicates(['Tile', 'geometry'], keep='last')
                self.delim_burrows = self.delim_burrows.drop_duplicates(['geometry'], keep='first')
                #if self.df_bboxes.loc[self.i, 'ID'] not in self.delim_burrows['Tile'].values:
                #    self.burrow_poly_dict[self.df_bboxes.loc[self.i, 'ID']] = hv.Polygons([])
        
        if self.delim_burrows.size > 0:
            self.delim_burrows.to_file('train_polys/delim_burrows_' + self.select_trainer.value + '.shp')
    
    @pm.depends('select_tile.param')
    def view(self):
        if self.burrows_annotate is not None:
            if len(self.burrows_annotate.annotated.data) > 0:
                self.burrow_poly_dict[self.df_bboxes.loc[self.i, 'ID']] = self.burrows_annotate.annotated
                for i in range(len(self.burrow_poly_dict[self.df_bboxes.loc[self.i, 'ID']].data)):
                    self.burrow_poly_dict[
                        self.df_bboxes.loc[self.i, 'ID']].data[i]['Tile'] = self.tile_list[self.df_bboxes.index.get_loc(self.i)]
                    self.burrow_poly_dict[
                        self.df_bboxes.loc[self.i, 'ID']].data[i]['Trainer'] = self.select_trainer.value
                tile_burrows_tmp = gpd.GeoDataFrame(data=self.burrow_poly_dict[self.df_bboxes.loc[self.i, 'ID']].data)
                tile_burrows_tmp.set_geometry(tile_burrows_tmp.apply(lambda row: Polygon(zip(row['x'], row['y'])), axis=1), inplace=True)
                tile_burrows_tmp.set_crs(epsg='32613', inplace=True)
                tile_burrows_tmp = tile_burrows_tmp.drop(columns=['x', 'y'])
                self.delim_burrows = self.delim_burrows.append(tile_burrows_tmp).drop_duplicates(['Tile', 'geometry'], keep='last')
                self.delim_burrows = self.delim_burrows.drop_duplicates(['geometry'], keep='first')
                #if self.df_bboxes.loc[self.i, 'ID'] not in self.delim_burrows['Tile'].values:
                #    self.burrow_poly_dict[self.df_bboxes.loc[self.i, 'ID']] = hv.Polygons([])
        
        if self.delim_burrows.size > 0:
            self.delim_burrows.to_file('train_polys/delim_burrows_' + self.select_trainer.value + '.shp')
            
        #self.i = int(self.select_tile.value.split('_')[-1])
        self.i = self.tile_list_dict[self.select_tile.value]

        self.rgb = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'rgb.tif').astype('int')
        self.rgb_img = self.rgb.hvplot.rgb(x='x', y='y', bands='band',
                                               **self.map_args).opts(**self.map_opts)

        self.ndvi = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'ndvi.tif').squeeze()
        self.ndvi_img = self.ndvi.hvplot.image(x='x', y='y',
                                                         **self.map_args).opts(cmap='viridis',
                                                                               colorbar=False,
                                                                               **self.map_opts)

        self.shade = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'shade.tif').squeeze()
        self.tpi = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'tpi.tif').squeeze()
        self.shade_img = self.shade.hvplot.image(x='x', y='y', 
                                     **self.map_args).opts(cmap='gray',
                                                           clim=(210, 225),
                                                           colorbar=False,
                                                           **self.map_opts)
        self.tpi_img = self.tpi.hvplot.image(x='x', y='y',
                                                    **self.map_args).opts(cmap='turbo',
                                                                          colorbar=False,
                                                                          **self.map_opts)

        
        self.minx, self.miny, self.maxx, self.maxy = self.df_bboxes.loc[self.i, ['min_x', 'min_y', 'max_x', 'max_y']].values
        self.poly_tmp = hv.Polygons(hv.Bounds((self.minx, self.miny, self.maxx, self.maxy))).opts('Polygons',
                                                                                                  line_color='red',
                                                                                                  fill_color=None, 
                                                                                                  **self.map_opts)
        
        
        self._burrows = self.burrow_poly_dict[self.df_bboxes.loc[self.i, 'ID']].opts(**self.draw_poly_opts)#.link(self.rgb_img)
        
        self.burrows_annotate = hv.annotate.instance()

        self.burrows_layout = self.burrows_annotate(self._burrows, 
                                                    annotations=['Comment', 'Tile', 'Trainer'],
                                                    vertex_style=dict(alpha=0.4, color='red', size=4),
                                                    default_opts=dict(framewise=False))
        
        if self.df_bboxes.loc[self.i]['Type'] == 'burrows_active':
            return pn.Column(hv.annotate.compose(hv.Overlay([self.rgb_img,
                                       self.ndvi_img.apply.opts(alpha=self.ndvi_alpha.param.value,
                                                                clim=self.ndvi_range.param.value), 
                                       (self.shade_img.apply.opts(alpha=self.terrain_alpha.param.value) * self.tpi_img.apply.opts(alpha=self.tpi_alpha.param.value,
                                                                                                          clim=self.terrain_range.param.value)),
                                       self.poly_tmp]).collate(),
                                       self.p_ground_burrows.apply.opts(alpha=self.ground_alpha.param.value),
                                       self.p_ground_other.apply.opts(alpha=self.ground_alpha.param.value),
                                       self.burrows_layout).opts(toolbar='left').redim.range(x=(np.min(self.rgb_img.data['x']) + self.window_size_m*0.75,
                                                                                                np.max(self.rgb_img.data['x']) - self.window_size_m*0.75),
                                                                                             y=(np.min(self.rgb_img.data['y']) + self.window_size_m*0.75,
                                                                                                np.max(self.rgb_img.data['y']) - self.window_size_m*0.75)),
                             hv.Table(self.delim_burrows[['Comment', 'Tile', 'Trainer']]).opts(height=140))
        else:
            return pn.Column(hv.annotate.compose(hv.Overlay([self.rgb_img,
                                       self.ndvi_img.apply.opts(alpha=self.ndvi_alpha.param.value,
                                                                clim=self.ndvi_range.param.value), 
                                       (self.shade_img.apply.opts(alpha=self.terrain_alpha.param.value) * self.tpi_img.apply.opts(alpha=self.tpi_alpha.param.value,
                                                                                                          clim=self.terrain_range.param.value)),
                                       self.poly_tmp]).collate(),
                                       self.burrows_layout).opts(toolbar='left').redim.range(x=(np.min(self.rgb_img.data['x']) + self.window_size_m*0.75,
                                                                                                np.max(self.rgb_img.data['x']) - self.window_size_m*0.75),
                                                                                             y=(np.min(self.rgb_img.data['y']) + self.window_size_m*0.75,
                                                                                                np.max(self.rgb_img.data['y']) - self.window_size_m*0.75)),
                             hv.Table(self.delim_burrows[['Comment', 'Tile', 'Trainer']]).opts(height=140))
    
    def layout(self):
        print('running layout')
        return pn.Row(
            pn.Column(
                pn.Column('### Update data', self.radio_all,
                          self.select_trainer,
                          self.select_tile,
                          pn.Param(self.param, widgets={
                    'save_polys': pn.widgets.Button(name='Save polygons', width=self.slider_width)}, show_name=False),                                                       
                          background='WhiteSmoke', margin=(5, 5, 5, 5), width=self.slider_width+50),
                pn.Column(
                    pn.Column('### NDVI', self.ndvi_alpha, self.ndvi_range, background='WhiteSmoke', margin=(5, 5, 5, 5), width=self.slider_width+50),
                    pn.Column('### Terrain', self.terrain_alpha, self.terrain_range, background='WhiteSmoke', margin=(5, 5, 5, 5), width=self.slider_width+50),
                    pn.Column('### Ground data', self.ground_alpha, background='WhiteSmoke', margin=(5, 5, 5, 5), width=self.slider_width+50)
                ), background='grey'),
            self.view)
    
    def layout2(self):
        print('running layout2')
        return pn.Row(
            pn.Column(
                pn.Column('### Update data', self.radio_all,
                          self.select_trainer,
                          self.select_tile,
                          pn.Param(self.param, widgets={
                    'save_polys': pn.widgets.Button(name='Save polygons', width=self.slider_width)}, show_name=False),                                                       
                          background='WhiteSmoke', margin=(5, 5, 5, 5), width=self.slider_width+50),
                pn.Column(
                    pn.Column('### NDVI', self.ndvi_alpha, self.ndvi_range, background='WhiteSmoke', margin=(5, 5, 5, 5), width=self.slider_width+50),
                    pn.Column('### Terrain', self.terrain_alpha, self.terrain_range, background='WhiteSmoke', margin=(5, 5, 5, 5), width=self.slider_width+50),
                    pn.Column('### Ground data', self.ground_alpha, background='WhiteSmoke', margin=(5, 5, 5, 5), width=self.slider_width+50)
                ), background='grey'))

In [ ]:
app = TrainApp()
print('app created')

In [ ]:
app.layout2().servable()